<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

# Financial Packages

&copy; Dr. Yves J. Hilpisch | The Python Quants GmbH

http://tpq.io | [training@tpq.io](mailto:trainin@tpq.io) | [@dyjh](http://twitter.com/dyjh)

### Please use the "Python 3.10, Numpy 1.26.4 Pandas TA" kernel.

## `vectorbt` package

In [ ]:
!git clone https://github.com/tpq-classes/financial_packages.git
import sys
sys.path.append('financial_packages')


In [ ]:
import numpy as np
import pandas as pd
from pylab import plt
plt.style.use('seaborn-v0_8')
%config InlineBackend.figure_format = 'svg'

In [ ]:
import warnings
warnings.simplefilter('ignore')

## Import

In [ ]:
import vectorbt as vbt

## Data

In [ ]:
url = 'https://certificate.tpq.io/findata.csv'

In [ ]:
raw = pd.read_csv(url, index_col=0, parse_dates=True)

In [ ]:
raw.info()

In [ ]:
#ticker = 'MSFT.O'
#ticker = 'AAPL.O'
ticker = 'GLD'
#ticker = '.SPX'

In [ ]:
data = pd.DataFrame(raw[ticker]).dropna()

In [ ]:
data.info()

In [ ]:
data.plot();

## From "Any" Signal

1 = long signal | -1 = short signal

In [ ]:
signals = np.array(sum(15 * [150 * [1] + 100 * [-1]], []))[:len(data)]
len(signals)

In [ ]:
signals[100:200]

In [ ]:
entries = np.where(signals == 1, True, False)
short_entries = np.where(signals == -1, True, False)

In [ ]:
entries = pd.DataFrame(entries, index=data.index)
short_entries = pd.DataFrame(short_entries, index=data.index)

In [ ]:
pf = vbt.Portfolio.from_signals(data, entries=entries,
                                short_entries=short_entries)

In [ ]:
pf.orders.count()

In [ ]:
pf[ticker].plot_orders()

In [ ]:
# pd.DataFrame.from_records(pf.order_records)

In [ ]:
pf.total_return()

In [ ]:
pf.total_benchmark_return()

In [ ]:
pf.wrapper.columns

In [ ]:
pf[ticker].plot().show()

### With Transaction Costs

#### Variable Fees

In [ ]:
# vbt.Portfolio.from_signals?

In [ ]:
# vbt.Portfolio.from_orders?

In [ ]:
pf = vbt.Portfolio.from_signals(data, entries=entries,
                                short_entries=short_entries,
                                fees=0.005, init_cash=10000)

In [ ]:
pf.orders.count()

In [ ]:
pf.total_return()

In [ ]:
pf.total_benchmark_return()

In [ ]:
pf.stats()

In [ ]:
pf.wrapper.columns

In [ ]:
pf[ticker].plot().show()

#### Fixed Fees

In [ ]:
pf = vbt.Portfolio.from_signals(data, entries=entries,
                                short_entries=short_entries,
                                fees=0.0025, fixed_fees=10.0,
                                init_cash=10000)

In [ ]:
pf.orders.count()

In [ ]:
pf.total_return()

In [ ]:
pf.total_benchmark_return()

In [ ]:
pf.stats()

In [ ]:
pf.wrapper.columns

In [ ]:
pf[ticker].plot().show()

## Stop Loss Orders

In [ ]:
pf = vbt.Portfolio.from_signals(data, entries=entries,
                                short_entries=short_entries,
                                sl_stop=0.04, init_cash=10000)

In [ ]:
pf.orders.count()

In [ ]:
pf.total_return()

In [ ]:
pf.total_benchmark_return()

In [ ]:
pf.stats()

In [ ]:
pf[ticker].plot().show()

## Trailing Stop Loss Orders

In [ ]:
pf = vbt.Portfolio.from_signals(data, entries=entries,
                                short_entries=short_entries,
                                sl_stop=0.04, sl_trail=True,
                                init_cash=10000)

In [ ]:
pf.orders.count()

In [ ]:
# pf[ticker].plot_orders()

In [ ]:
# pd.DataFrame.from_records(pf.order_records)

In [ ]:
pf.total_return()

In [ ]:
pf.total_benchmark_return()

In [ ]:
pf.stats()

In [ ]:
pf[ticker].plot().show()

## Take Profit Orders

In [ ]:
pf = vbt.Portfolio.from_signals(data, entries=entries,
                                short_entries=short_entries,
                                tp_stop=0.06, init_cash=10000)

In [ ]:
pf.orders.count()

In [ ]:
pf.total_return()

In [ ]:
pf.total_benchmark_return()

In [ ]:
pf.stats()

In [ ]:
pf[ticker].plot().show()

## Slippage

In [ ]:
np.linspace(0., 0.0025, 10)

In [ ]:
pf = vbt.Portfolio.from_signals(data, entries=entries,
                                short_entries=short_entries,
                                slippage=np.linspace(0., 0.0025, 10),
                                init_cash=10000)

In [ ]:
pf.total_return()

## All Together

In [ ]:
data = raw[[ticker, 'GDX']].dropna()

In [ ]:
sl_stop = 0.075
sl_trail = False
tp_stop = np.inf
fees = 0.0025
fixed_fees = 0
slippage = 0.0005
init_cash = 10000

In [ ]:
pf = vbt.Portfolio.from_signals(data, entries=entries,
                                short_entries=short_entries,
                                sl_stop=sl_stop, sl_trail=sl_trail,
                                tp_stop=tp_stop,
                                fees=fees, fixed_fees=fixed_fees,
                                slippage=slippage,
                                init_cash=init_cash)

In [ ]:
pf.orders.count()

In [ ]:
# pd.DataFrame.from_records(pf[data.columns[0]].order_records)

In [ ]:
pf.total_return()

In [ ]:
pf.total_benchmark_return()

In [ ]:
pf[data.columns[1]].stats()

In [ ]:
pf.wrapper.columns

In [ ]:
pf[data.columns[1]].plot().show()

## Large Scale Backtesting

In [ ]:
from numpy.random import default_rng

In [ ]:
rng = default_rng()

In [ ]:
data = raw.dropna()  # all 12 instruments

In [ ]:
N = 100  # number of random strategy configurations

In [ ]:
len(data.columns) * N  # number of strategies backtested

In [ ]:
sma1 = rng.integers(10, 30, N)

In [ ]:
sma2 = rng.integers(100, 300, N)

In [ ]:
%%time
SMA1 = vbt.MA.run(data, sma1, short_name='fast')
SMA2 = vbt.MA.run(data, sma2, short_name='slow')
entries = SMA1.ma_crossed_above(SMA2)
short_entries = SMA1.ma_crossed_below(SMA2)
pf = vbt.Portfolio.from_signals(data, entries=entries,
                                short_entries=short_entries)

In [ ]:
%time pf.total_return()

In [ ]:
%time pf.drawdowns.max_drawdown()

In [ ]:
%time pf.drawdowns.max_duration()

In [ ]:
top = pf.total_return().sort_values(ascending=False)

In [ ]:
top.head(10)

In [ ]:
i = top.index[0]

In [ ]:
pf[i].stats()

In [ ]:
# pf[i].plot().show()  # seems to work only for N <= 100

<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

<a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="mailto:training@tpq.io">training@tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> 